In [1]:

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

In [2]:
import zipfile
zip_ref = zipfile.ZipFile('/content/Deepdive.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [3]:
#generators
train_ds=keras.utils.image_dataset_from_directory(
    directory='/content/train',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256,256)
)

test_ds=keras.utils.image_dataset_from_directory(
    directory='/content/test',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256,256)
)
validation_ds=keras.utils.image_dataset_from_directory(
    directory='/content/validation',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256,256)
)

Found 2667 files belonging to 33 classes.
Found 660 files belonging to 33 classes.
Found 667 files belonging to 33 classes.


In [5]:
print(train_ds)


<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [6]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = "/content/train"
test_dir = "/content/test"
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

test_datagen = ImageDataGenerator(
    rescale=1.0/255
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

print("Train Classes:", train_generator.class_indices)
print("Test Classes:", test_generator.class_indices)

images, labels = next(train_generator)
print("Batch Shape (images):", images.shape)
print("Batch Shape (labels):", labels.shape)

Found 2667 images belonging to 33 classes.
Found 660 images belonging to 33 classes.
Train Classes: {'Anthomastus Like coral': 0, 'Arborescent Stumpy Grey Branching Erect sponges': 1, 'Arborescent coral': 2, 'Attached Stony coral': 3, 'Bioturbation - Organism tests': 4, 'Bony fishes': 5, 'Colonial Unstalked Ascidian': 6, 'Crab': 7, 'Ctenophores jellies': 8, 'Cup-likes sponges': 9, 'Eels': 10, 'Feather stars': 11, 'Free-living coral': 12, 'General unknown biology': 13, 'Hermit Crab with Shell or Stone': 14, 'Irregular urchins': 15, 'Laminar Erect sponges': 16, 'Octopods': 17, 'Other anemones': 18, 'Prawns - Shrimps - Mysids': 19, 'Quill (Sea pen) coral': 20, 'Sea Cucumber': 21, 'Sea stars': 22, 'Sea urchins': 23, 'Sea whip coral': 24, 'Simple Branching coral': 25, 'Snake stars': 26, 'Squat lobsters': 27, 'Stalked Erect sponges': 28, 'True lobsters': 29, 'Tube anemones': 30, 'Tube worms': 31, 'Woody debris': 32}
Test Classes: {'Anthomastus Like coral': 0, 'Arborescent Stumpy Grey Branchi

#Resize image#

In [ ]:
import os
import cv2

def resize_images(input_dir, output_dir, target_size=(224, 224)):
    for root, _, files in os.walk(input_dir):
        for file in files:
                img_path = os.path.join(root, file)
                img = cv2.imread(img_path)
                if img is None:
                    print(f"Could not read image: {img_path}")
                    continue
                img_resized = cv2.resize(img, target_size)
                relative_path = os.path.relpath(root, input_dir)
                save_dir = os.path.join(output_dir, relative_path)
                os.makedirs(save_dir, exist_ok=True)
                save_path = os.path.join(save_dir, file)

                cv2.imwrite(save_path, img_resized)
                print(f"Resized and saved: {save_path}")

input_dir = "/content/train"
output_dir = "/content/train_set"

resize_images(input_dir, output_dir, target_size=(224, 224))

print("All images have been resized!")

#Rescale image#

In [18]:
def normalize_image(img):
    """Normalize pixel values to [0, 1]."""
    return img / 255.0

# Grayscale Conversion

In [9]:
def convert_to_grayscale(img):
    """Convert image to grayscale."""
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Denoising

In [10]:
def denoise_image(img):
    """Denoise image using Gaussian Blur."""
    return cv2.GaussianBlur(img, (5, 5), 0)

# Cropping

In [11]:
def crop_image(img, crop_size=(200, 200)):
    """Crop the center of the image to crop_size."""
    h, w = img.shape[:2]
    start_x = (w - crop_size[0]) // 2
    start_y = (h - crop_size[1]) // 2
    return img[start_y:start_y+crop_size[1], start_x:start_x+crop_size[0]]

# Histogram Equalization

In [12]:
def histogram_equalization(img):
    """Apply histogram equalization to enhance contrast."""
    if len(img.shape) == 2:  # Grayscale
        return cv2.equalizeHist(img)
    elif len(img.shape) == 3:  # Color
        ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
        ycrcb[:, :, 0] = cv2.equalizeHist(ycrcb[:, :, 0])
        return cv2.cvtColor(ycrcb, cv2.COLOR_YCrCb2BGR)

# Color Space Conversion

In [13]:
def convert_color_space(img, target_space=cv2.COLOR_BGR2HSV):
    """Convert image to a different color space (e.g., HSV)."""
    return cv2.cvtColor(img, target_space)

# Binarization/Thresholding

In [14]:
def binarize_image(img, threshold=128):
    """Binarize image using a threshold."""
    _, binary_img = cv2.threshold(img, threshold, 255, cv2.THRESH_BINARY)
    return binary_img

# Data Augmentation

In [15]:
def data_augmentation(img):
    """Apply basic data augmentation: horizontal flip, rotation."""
    augmented_images = []

    flipped = cv2.flip(img, 1)
    augmented_images.append(flipped)

    rows, cols = img.shape[:2]
    M = cv2.getRotationMatrix2D((cols / 2, rows / 2), 15, 1)
    rotated = cv2.warpAffine(img, M, (cols, rows))
    augmented_images.append(rotated)

    return augmented_images

# Preprocessing Pipeline

In [16]:
def preprocess_images(input_dir, output_dir, crop_size=(200, 200)):
    os.makedirs(output_dir, exist_ok=True)

    for root, _, files in os.walk(input_dir):
        for file in files:
            if file.endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(root, file)
                img = cv2.imread(img_path)
                img_normalized = normalize_image(img)
                img_gray = convert_to_grayscale(img)
                img_denoised = denoise_image(img)
                img_cropped = crop_image(img_denoised, crop_size)
                img_equalized = histogram_equalization(img_cropped)
                img_hsv = convert_color_space(img_equalized, cv2.COLOR_BGR2HSV)
                img_binary = binarize_image(img_gray)

                relative_path = os.path.relpath(root, input_dir)
                save_dir = os.path.join(output_dir, relative_path)
                os.makedirs(save_dir, exist_ok=True)

                cv2.imwrite(os.path.join(save_dir, f"{file}_normalized.jpg"), img_normalized * 255)
                cv2.imwrite(os.path.join(save_dir, f"{file}_gray.jpg"), img_gray)
                cv2.imwrite(os.path.join(save_dir, f"{file}_denoised.jpg"), img_denoised)
                cv2.imwrite(os.path.join(save_dir, f"{file}_cropped.jpg"), img_cropped)
                cv2.imwrite(os.path.join(save_dir, f"{file}_equalized.jpg"), img_equalized)
                cv2.imwrite(os.path.join(save_dir, f"{file}_hsv.jpg"), img_hsv)
                cv2.imwrite(os.path.join(save_dir, f"{file}_binary.jpg"), img_binary)

                augmented_images = data_augmentation(img_cropped)
                for i, aug_img in enumerate(augmented_images):
                    cv2.imwrite(os.path.join(save_dir, f"{file}_augmented_{i}.jpg"), aug_img)

                print(f"Processed: {img_path}")

In [19]:

input_dir = "/content/train_set"
output_dir = "/content/train_submission"

preprocess_images(input_dir, output_dir)
print("Image preprocessing completed!")

Processed: /content/train_set/Anthomastus Like coral/Anthomastus Like coral_603_image.jpg
Processed: /content/train_set/Anthomastus Like coral/Anthomastus Like coral_2619_image.jpg
Processed: /content/train_set/Anthomastus Like coral/Anthomastus Like coral_435_image.jpg
Processed: /content/train_set/Anthomastus Like coral/Anthomastus Like coral_109_image.jpg
Processed: /content/train_set/Anthomastus Like coral/Anthomastus Like coral_1100_image.jpg
Processed: /content/train_set/Anthomastus Like coral/Anthomastus Like coral_290_image.jpg
Processed: /content/train_set/Anthomastus Like coral/Anthomastus Like coral_1033_image.jpg
Processed: /content/train_set/Anthomastus Like coral/Anthomastus Like coral_2134_image.jpg
Processed: /content/train_set/Anthomastus Like coral/Anthomastus Like coral_393_image.jpg
Processed: /content/train_set/Anthomastus Like coral/Anthomastus Like coral_1395_image.jpg
Processed: /content/train_set/Anthomastus Like coral/Anthomastus Like coral_1786_image.jpg
Proc